# 04_InteractionDiscovery - Physics-SR Framework v4.1

## Stage 1.4: EBM-Guided Interaction Discovery with Bootstrap Stability

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Date:** January 2026  
**Version:** 4.1.1 (EBM-based Interaction Discovery - 75x faster than iRF)

---

### Purpose

Discover high-order feature interactions using Explainable Boosting Machine (EBM) with bootstrap stability validation. This replaces the original iRF approach for dramatic speed improvements.

### Key Innovation (v4.1.1)

**Previous iRF Problem:**
- Exponential complexity in feature count
- 3 features: 14 seconds, 8 features: 1530 seconds (25 minutes!)

**EBM Solution:**
- Uses FAST algorithm for O(p^2) pairwise interaction detection
- Cyclic boosting naturally handles correlated features
- 8 features: ~10-20 seconds (75x speedup)

### Algorithm

1. Fit EBM with automatic pairwise interaction detection
2. Extract detected interactions and their importances
3. Bootstrap stability assessment (50 iterations)
4. Filter interactions by stability threshold

### Reference

- Lou et al. (2013). Accurate Intelligible Models with Pairwise Interactions. KDD.
- Nori et al. (2019). InterpretML: A Unified Framework for Machine Learning Interpretability.

---
## Section 1: Header and Imports

In [ ]:
"""
04_InteractionDiscovery.ipynb - EBM-Guided Interaction Discovery
=================================================================

Three-Stage Physics-Informed Symbolic Regression Framework v4.1.1

This module provides:
- IRFInteractionDiscoverer: Discover feature interactions using EBM (backward compatible name)
- Bootstrap stability assessment for robust interaction identification
- 75x speedup over original iRF implementation

Algorithm (EBM + Bootstrap):
    1. Fit Explainable Boosting Machine with pairwise interactions
    2. Extract detected interactions from EBM term names
    3. Bootstrap stability assessment (50+ iterations)
    4. Filter by stability threshold (default 0.6)

Output Dictionary Keys (v4.1 compatible):
    - soft_weights: Dict of feature importance weights
    - selected_features: List of features above selection threshold
    - stable_interactions: List of bootstrap-stable interactions
    - interaction_stability: Dict mapping interactions to stability scores
    - raw_importance: Dict of raw feature importance
    - suggested_terms: List of feature product strings for library

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
Contact: zz3239@columbia.edu
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Interaction Discovery
from collections import Counter
from itertools import combinations
from typing import Dict, List, Tuple, Optional, Any, Set, FrozenSet
from sklearn.utils import resample

# EBM import with fallback
try:
    from interpret.glassbox import ExplainableBoostingRegressor
    EBM_AVAILABLE = True
    print("04_InteractionDiscovery v4.1.1: EBM import successful.")
except ImportError:
    EBM_AVAILABLE = False
    print("WARNING: interpret package not found. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'interpret', '-q'])
    from interpret.glassbox import ExplainableBoostingRegressor
    EBM_AVAILABLE = True
    print("04_InteractionDiscovery v4.1.1: EBM installed and imported.")

# Fallback: RandomForest for backup
from sklearn.ensemble import RandomForestRegressor

print("04_InteractionDiscovery v4.1.1: All imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# IRF INTERACTION DISCOVERER CLASS (EBM-BASED v4.1.1)
# ==============================================================================

class IRFInteractionDiscoverer:
    """
    EBM-Guided Interaction Discovery with Bootstrap Stability.
    
    This discoverer identifies pairwise feature interactions by:
    1. Training Explainable Boosting Machine (EBM) with automatic interaction detection
    2. Extracting interaction terms from EBM's learned structure
    3. Using bootstrap stability to filter genuine interactions
    
    v4.1.1 UPGRADE: Replaces iRF with EBM for 75x speedup while maintaining
    stability guarantees through explicit bootstrap validation.
    
    The class name is kept as 'IRFInteractionDiscoverer' for backward compatibility.
    
    Attributes
    ----------
    temperature : float
        Softmax temperature parameter (kept for API compatibility)
    selection_threshold : float
        Minimum importance for feature selection (default: 0.1)
    n_bootstrap : int
        Number of bootstrap samples for stability assessment (default: 50)
    stability_threshold : float
        Minimum bootstrap frequency for stable interaction (default: 0.6)
    max_interactions : int
        Maximum number of interactions for EBM to detect (default: 15)
    
    Methods
    -------
    discover(X, y, feature_names) -> Dict
        Discover stable pairwise interactions
    get_stable_interactions() -> List[Tuple[str, ...]]
        Get list of stable interactions as tuples
    get_interaction_matrix(X) -> Tuple[np.ndarray, List[str]]
        Compute interaction features from stable interactions
    print_interaction_report() -> None
        Print detailed discovery report
    
    Reference
    ---------
    Lou et al. (2013). KDD - FAST algorithm for pairwise interactions.
    
    Examples
    --------
    >>> discoverer = IRFInteractionDiscoverer(n_bootstrap=50)
    >>> result = discoverer.discover(X, y, feature_names)
    >>> print(result['stable_interactions'])
    [('x0', 'x1'), ('x1', 'x2')]  # Discovered interactions
    """
    
    def __init__(
        self,
        temperature: float = DEFAULT_SOFTMAX_TEMPERATURE,
        selection_threshold: float = DEFAULT_IMPORTANCE_THRESHOLD,
        n_bootstrap: int = 50,
        stability_threshold: float = 0.6,  # Higher default for EBM
        max_interaction_order: int = 2,  # EBM only supports pairwise
        max_interactions: int = 15,
        n_estimators: int = 200,  # Kept for API compatibility
        random_state: int = RANDOM_SEED
    ):
        """
        Initialize IRFInteractionDiscoverer (EBM-based v4.1.1).
        
        Parameters
        ----------
        temperature : float
            Softmax temperature (kept for API compatibility).
        selection_threshold : float
            Minimum importance for a feature to be considered.
        n_bootstrap : int
            Number of bootstrap samples for stability assessment.
        stability_threshold : float
            Minimum bootstrap frequency for an interaction to be stable.
        max_interaction_order : int
            Maximum interaction order (EBM supports up to 2).
        max_interactions : int
            Maximum number of interactions for EBM to detect.
        n_estimators : int
            Kept for API compatibility (not used by EBM).
        random_state : int
            Random seed for reproducibility.
        """
        self.temperature = temperature
        self.selection_threshold = selection_threshold
        self.n_bootstrap = n_bootstrap
        self.stability_threshold = stability_threshold
        self.max_interaction_order = min(max_interaction_order, 2)  # EBM limit
        self.max_interactions = max_interactions
        self.n_estimators = n_estimators
        self.random_state = random_state
        
        # Internal state
        self._feature_names = None
        self._ebm_model = None
        self._raw_importance = None
        self._soft_weights = None
        self._selected_features = None
        self._all_interactions = None
        self._interaction_stability = None
        self._stable_interactions = None
        self._discovery_complete = False
    
    def discover(
        self,
        X: np.ndarray,
        y: np.ndarray,
        feature_names: List[str]
    ) -> Dict[str, Any]:
        """
        Discover feature interactions using EBM approach.
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix of shape (n_samples, n_features)
        y : np.ndarray
            Target vector of shape (n_samples,)
        feature_names : List[str]
            Names of features corresponding to columns of X
        
        Returns
        -------
        Dict[str, Any]
            Dictionary containing (v4.1 compatible keys):
            - soft_weights: Dict of importance weights
            - selected_features: List of features above selection threshold
            - stable_interactions: List of bootstrap-stable interactions
            - interaction_stability: Dict mapping interactions to stability scores
            - raw_importance: Dict of raw importance
            - all_interactions: List of all discovered interactions
            - suggested_terms: List of feature product strings for library
            - n_stable_interactions: Number of stable interactions
        """
        self._feature_names = list(feature_names)
        n_features = X.shape[1]
        
        # Step 1: Fit EBM and extract interactions
        self._ebm_model, ebm_interactions = self._fit_ebm(X, y)
        
        # Step 2: Extract feature importance from EBM
        self._raw_importance, self._soft_weights = self._extract_importance()
        
        # Step 3: Select features above threshold
        self._selected_features = [
            name for name, weight in self._soft_weights.items()
            if weight > self.selection_threshold
        ]
        
        # Step 4: Store all detected interactions
        self._all_interactions = set(
            frozenset(pair) for pair in ebm_interactions
        )
        
        # Step 5: Bootstrap stability assessment
        self._interaction_stability = self._bootstrap_stability(X, y)
        
        # Step 6: Filter stable interactions
        self._stable_interactions = [
            interaction for interaction, stability 
            in self._interaction_stability.items()
            if stability >= self.stability_threshold
        ]
        
        self._discovery_complete = True
        
        # Build suggested terms for feature library
        suggested_terms = self._build_suggested_terms()
        
        return {
            # v4.1 primary keys
            'soft_weights': self._soft_weights,
            'selected_features': self._selected_features,
            'stable_interactions': self._stable_interactions,
            'interaction_stability': dict(self._interaction_stability),
            # Additional useful keys
            'raw_importance': self._raw_importance,
            'all_interactions': list(self._all_interactions),
            'suggested_terms': suggested_terms,
            'n_stable_interactions': len(self._stable_interactions),
            'temperature': self.temperature,
            'stability_threshold': self.stability_threshold,
            # Backward compatibility alias
            'softmax_weights': self._soft_weights
        }
    
    def _fit_ebm(
        self,
        X: np.ndarray,
        y: np.ndarray
    ) -> Tuple[Any, List[Tuple[str, str]]]:
        """
        Fit EBM and extract detected interactions.
        
        Returns
        -------
        Tuple[EBM model, List of interaction pairs]
        """
        ebm = ExplainableBoostingRegressor(
            interactions=self.max_interactions,
            max_bins=256,
            max_rounds=5000,
            learning_rate=0.01,
            random_state=self.random_state,
            n_jobs=-1
        )
        ebm.fit(X, y, feature_names=self._feature_names)
        
        # Extract interaction pairs from term names
        interactions = []
        for term in ebm.term_names_:
            if ' x ' in term:  # EBM interaction format: "feature1 x feature2"
                parts = term.split(' x ')
                if len(parts) == 2:
                    f1, f2 = parts[0].strip(), parts[1].strip()
                    if f1 in self._feature_names and f2 in self._feature_names:
                        interactions.append((f1, f2))
        
        return ebm, interactions
    
    def _extract_importance(
        self
    ) -> Tuple[Dict[str, float], Dict[str, float]]:
        """
        Extract feature importance from EBM.
        
        Returns
        -------
        Tuple[raw_importance dict, soft_weights dict]
        """
        # Get term importances from EBM
        term_names = self._ebm_model.term_names_
        term_importances = self._ebm_model.term_importances_
        
        # Aggregate importance for each feature (main effects only)
        raw_importance = {name: 0.0 for name in self._feature_names}
        
        for term, importance in zip(term_names, term_importances):
            if ' x ' not in term:  # Main effect only
                if term in raw_importance:
                    raw_importance[term] = float(importance)
        
        # Apply softmax transformation for soft_weights
        importance_array = np.array([raw_importance[name] for name in self._feature_names])
        
        # Softmax with temperature
        if np.max(importance_array) > 0:
            scaled = importance_array / self.temperature
            scaled = scaled - np.max(scaled)  # Numerical stability
            exp_scaled = np.exp(scaled)
            weights = exp_scaled / np.sum(exp_scaled)
        else:
            weights = np.ones(len(self._feature_names)) / len(self._feature_names)
        
        soft_weights = {
            name: float(weights[i])
            for i, name in enumerate(self._feature_names)
        }
        
        return raw_importance, soft_weights
    
    def _bootstrap_stability(
        self,
        X: np.ndarray,
        y: np.ndarray
    ) -> Dict[FrozenSet[str], float]:
        """
        Assess interaction stability via bootstrap resampling.
        
        For each bootstrap sample, fit EBM and extract interactions.
        Stability = frequency of interaction across bootstrap samples.
        
        Parameters
        ----------
        X : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        
        Returns
        -------
        Dict[FrozenSet[str], float]
            Dictionary mapping interactions to stability scores
        """
        interaction_counts = Counter()
        
        for b in range(self.n_bootstrap):
            # Bootstrap resample
            X_boot, y_boot = resample(
                X, y, 
                random_state=self.random_state + b if self.random_state else None
            )
            
            # Fit EBM on bootstrap sample (faster settings)
            try:
                ebm_boot = ExplainableBoostingRegressor(
                    interactions=self.max_interactions,
                    max_bins=128,  # Faster
                    max_rounds=2000,  # Faster
                    learning_rate=0.02,
                    random_state=self.random_state + b if self.random_state else None,
                    n_jobs=-1
                )
                ebm_boot.fit(X_boot, y_boot, feature_names=self._feature_names)
                
                # Extract interactions
                for term in ebm_boot.term_names_:
                    if ' x ' in term:
                        parts = term.split(' x ')
                        if len(parts) == 2:
                            f1, f2 = parts[0].strip(), parts[1].strip()
                            if f1 in self._feature_names and f2 in self._feature_names:
                                # Normalize order for consistent counting
                                pair = frozenset([f1, f2])
                                interaction_counts[pair] += 1
            except Exception as e:
                # Skip failed bootstrap iterations
                continue
        
        # Compute stability scores
        stability_scores = {
            interaction: count / self.n_bootstrap
            for interaction, count in interaction_counts.items()
        }
        
        return stability_scores
    
    def _build_suggested_terms(
        self
    ) -> List[str]:
        """
        Build suggested interaction terms for feature library.
        
        Returns
        -------
        List[str]
            List of feature product strings (e.g., "x0*x1")
        """
        terms = []
        for interaction in self._stable_interactions:
            sorted_features = sorted(interaction)
            term = " * ".join(sorted_features)
            terms.append(term)
        return terms
    
    def get_stable_interactions(
        self
    ) -> List[Tuple[str, ...]]:
        """
        Get list of stable interactions as tuples.
        
        Returns
        -------
        List[Tuple[str, ...]]
            List of stable interactions
        
        Raises
        ------
        RuntimeError
            If discovery has not been performed
        """
        if not self._discovery_complete:
            raise RuntimeError("Must run discover() before getting interactions")
        
        return [tuple(sorted(interaction)) 
                for interaction in self._stable_interactions]
    
    def get_interaction_matrix(
        self,
        X: np.ndarray
    ) -> Tuple[np.ndarray, List[str]]:
        """
        Compute interaction features from stable interactions.
        
        Parameters
        ----------
        X : np.ndarray
            Original feature matrix
        
        Returns
        -------
        Tuple[np.ndarray, List[str]]
            - Interaction feature matrix
            - Names of interaction features
        """
        if not self._discovery_complete:
            raise RuntimeError("Must run discover() before computing interaction matrix")
        
        if len(self._stable_interactions) == 0:
            return np.empty((X.shape[0], 0)), []
        
        n_samples = X.shape[0]
        interaction_features = []
        interaction_names = []
        
        # Create mapping from feature name to column index
        name_to_idx = {name: i for i, name in enumerate(self._feature_names)}
        
        for interaction in self._stable_interactions:
            # Compute product of features in interaction
            product = np.ones(n_samples)
            sorted_features = sorted(interaction)
            
            for feat_name in sorted_features:
                idx = name_to_idx[feat_name]
                product *= X[:, idx]
            
            interaction_features.append(product)
            interaction_names.append("*".join(sorted_features))
        
        return np.column_stack(interaction_features), interaction_names
    
    def print_interaction_report(self) -> None:
        """
        Print a detailed interaction discovery report.
        """
        if not self._discovery_complete:
            print("Discovery not yet performed. Run discover() first.")
            return
        
        print("=" * 70)
        print("=== Interaction Discovery Results (EBM v4.1.1) ===")
        print("=" * 70)
        print()
        print("Configuration:")
        print(f"  Method: Explainable Boosting Machine (EBM)")
        print(f"  Stability threshold: {self.stability_threshold}")
        print(f"  Bootstrap samples: {self.n_bootstrap}")
        print(f"  Max interactions: {self.max_interactions}")
        print()
        print("-" * 70)
        print(" Feature Importance:")
        print("-" * 70)
        print(f"{'Feature':<20} {'Raw Importance':<15} {'Soft Weight':<15} {'Selected'}")
        print("-" * 70)
        
        # Sort by soft weight
        sorted_features = sorted(
            self._soft_weights.items(),
            key=lambda x: x[1],
            reverse=True
        )
        
        for name, soft in sorted_features:
            raw = self._raw_importance.get(name, 0)
            selected = "YES" if name in self._selected_features else "no"
            print(f"{name:<20} {raw:<15.4f} {soft:<15.4f} {selected}")
        
        print()
        print("-" * 70)
        print(" Stable Interactions:")
        print("-" * 70)
        
        if len(self._stable_interactions) == 0:
            print("  No stable interactions found.")
        else:
            # Sort by stability
            sorted_interactions = sorted(
                self._stable_interactions,
                key=lambda x: self._interaction_stability.get(x, 0),
                reverse=True
            )
            
            print(f"{'Interaction':<30} {'Stability Score':<15}")
            print("-" * 50)
            for interaction in sorted_interactions:
                name = " * ".join(sorted(interaction))
                stability = self._interaction_stability.get(interaction, 0)
                print(f"{name:<30} {stability:<15.3f}")
        
        print()
        print(f"Total stable interactions: {len(self._stable_interactions)}")
        print(f"Total candidate interactions: {len(self._all_interactions)}")
        print()
        print("=" * 70)

print("IRFInteractionDiscoverer class (EBM-based v4.1.1) defined.")

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 04_InteractionDiscovery v4.1.1 (EBM)")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Known Interactions
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Known Interactions")
    
    # Generate data with known interactions
    np.random.seed(42)
    n_samples = 500
    
    x0 = np.random.uniform(0, 1, n_samples)
    x1 = np.random.uniform(0, 1, n_samples)
    x2 = np.random.uniform(0, 1, n_samples)
    x3 = np.random.randn(n_samples)  # Noise feature
    
    # True equation: y = 3*x0*x1 + x2^2 + noise
    # Interaction: (x0, x1)
    y = 3 * x0 * x1 + x2**2 + 0.1 * np.random.randn(n_samples)
    
    X = np.column_stack([x0, x1, x2, x3])
    feature_names = ['x0', 'x1', 'x2', 'x3']
    
    print(f"True equation: y = 3*x0*x1 + x2^2")
    print(f"Expected interaction: (x0, x1)")
    print()
    
    import time
    start_time = time.time()
    
    # Run discovery
    discoverer = IRFInteractionDiscoverer(
        n_bootstrap=30,  # Reduced for faster testing
        stability_threshold=0.4
    )
    result = discoverer.discover(X, y, feature_names)
    
    elapsed = time.time() - start_time
    print(f"Discovery completed in {elapsed:.2f} seconds")
    print()
    
    # Verify output keys match v4.1 specification
    print("Output Dictionary Keys:")
    for key in result.keys():
        print(f"  {key}")
    print()
    
    # Print report
    discoverer.print_interaction_report()
    
    # Verification
    print("\nVerification:")
    stable_interactions = discoverer.get_stable_interactions()
    
    x0_x1_found = any(
        set(interaction) == {'x0', 'x1'}
        for interaction in stable_interactions
    )
    
    if x0_x1_found:
        print("  [PASS] Correctly identified (x0, x1) interaction")
    else:
        print("  [WARNING] Did not identify (x0, x1) interaction")
        print(f"  Found interactions: {stable_interactions}")

In [ ]:
# ==============================================================================
# TEST 2: Speed Comparison (EBM vs RF reference)
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: Speed Test with 8 Features")
    
    np.random.seed(42)
    n_samples = 500
    n_features = 8  # The problematic case for iRF
    
    X = np.random.uniform(0, 1, (n_samples, n_features))
    # Create interaction: x0*x1 + x2*x3
    y = 2 * X[:, 0] * X[:, 1] + 3 * X[:, 2] * X[:, 3] + 0.1 * np.random.randn(n_samples)
    
    feature_names = [f'x{i}' for i in range(n_features)]
    
    print(f"Testing with {n_features} features (this would take ~25 min with iRF)")
    print()
    
    import time
    start_time = time.time()
    
    discoverer = IRFInteractionDiscoverer(
        n_bootstrap=50,
        stability_threshold=0.5
    )
    result = discoverer.discover(X, y, feature_names)
    
    elapsed = time.time() - start_time
    
    print(f"EBM Discovery completed in {elapsed:.2f} seconds")
    print(f"(Original iRF would take ~1530 seconds)")
    print(f"Speedup: ~{1530/elapsed:.0f}x")
    print()
    
    print(f"Stable interactions found: {len(result['stable_interactions'])}")
    for interaction in result['stable_interactions']:
        name = ' * '.join(sorted(interaction))
        stability = result['interaction_stability'].get(interaction, 0)
        print(f"  {name}: stability={stability:.3f}")

In [ ]:
# ==============================================================================
# TEST 3: Bootstrap Stability Verification
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: Bootstrap Stability Verification")
    
    # Strong interaction should have high stability
    # Spurious interaction should have low stability
    np.random.seed(42)
    n_samples = 500
    
    x0 = np.random.uniform(0, 1, n_samples)
    x1 = np.random.uniform(0, 1, n_samples)
    x2 = np.random.uniform(0, 1, n_samples)  # Independent
    
    # Strong interaction: x0*x1
    y = 5 * x0 * x1 + 0.1 * np.random.randn(n_samples)
    
    X = np.column_stack([x0, x1, x2])
    feature_names = ['x0', 'x1', 'x2']
    
    print(f"True equation: y = 5*x0*x1")
    print(f"x2 is independent (no true interaction with x0 or x1)")
    print()
    
    discoverer = IRFInteractionDiscoverer(
        n_bootstrap=50,
        stability_threshold=0.3
    )
    result = discoverer.discover(X, y, feature_names)
    
    print(f"Interaction Stability Scores:")
    for interaction, stability in sorted(
        result['interaction_stability'].items(),
        key=lambda x: x[1],
        reverse=True
    ):
        name = " * ".join(sorted(interaction))
        print(f"  {name}: {stability:.3f}")
    
    # Check x0*x1 has highest stability
    print()
    x0_x1_stability = result['interaction_stability'].get(
        frozenset(['x0', 'x1']), 0
    )
    
    if x0_x1_stability > 0.5:
        print(f"  [PASS] x0*x1 has high stability ({x0_x1_stability:.3f})")
    else:
        print(f"  [WARNING] x0*x1 stability is low ({x0_x1_stability:.3f})")

In [ ]:
# ==============================================================================
# TEST 4: API Compatibility Check
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: API Compatibility Check")
    
    np.random.seed(42)
    X = np.random.uniform(0, 1, (100, 3))
    y = X[:, 0] * X[:, 1] + X[:, 2]
    feature_names = ['a', 'b', 'c']
    
    discoverer = IRFInteractionDiscoverer(n_bootstrap=10)
    result = discoverer.discover(X, y, feature_names)
    
    # Check all v4.1 required keys exist
    required_keys = [
        'soft_weights',
        'selected_features', 
        'stable_interactions',
        'interaction_stability',
        'raw_importance',
        'all_interactions',
        'suggested_terms',
        'n_stable_interactions',
        'temperature',
        'stability_threshold',
        'softmax_weights'  # Backward compatibility
    ]
    
    print("Checking v4.1 API compatibility:")
    all_pass = True
    for key in required_keys:
        if key in result:
            print(f"  [PASS] {key}")
        else:
            print(f"  [FAIL] {key} missing!")
            all_pass = False
    
    # Check methods exist
    print("\nChecking required methods:")
    methods = ['discover', 'get_stable_interactions', 'get_interaction_matrix', 'print_interaction_report']
    for method in methods:
        if hasattr(discoverer, method):
            print(f"  [PASS] {method}()")
        else:
            print(f"  [FAIL] {method}() missing!")
            all_pass = False
    
    if all_pass:
        print("\n[SUCCESS] Full v4.1 API compatibility confirmed.")

In [ ]:
# ==============================================================================
# TEST SUMMARY
# ==============================================================================

if _RUN_TESTS:
    print()
    print("=" * 70)
    print(" INTERNAL TESTS COMPLETE")
    print("=" * 70)
    print()
    print("Key improvements in v4.1.1:")
    print("  1. EBM replaces iRF for ~75x speedup")
    print("  2. Bootstrap stability validation maintained")
    print("  3. Full backward compatibility with v4.1 API")
    print("  4. 8 features: ~20s vs 1530s with original iRF")

---
## Section 4: Module Export

In [ ]:
# ==============================================================================
# MODULE COMPLETE
# ==============================================================================

print("=" * 70)
print(" 04_InteractionDiscovery v4.1.1 (EBM-based) loaded successfully.")
print("=" * 70)
print()
print("Exported:")
print("  - IRFInteractionDiscoverer (EBM-based, backward compatible name)")
print()
print("Key Features:")
print("  - 75x speedup over original iRF implementation")
print("  - Bootstrap stability validation for robust interactions")
print("  - Full v4.1 API compatibility")
print()
print("Usage:")
print("  discoverer = IRFInteractionDiscoverer(n_bootstrap=50)")
print("  result = discoverer.discover(X, y, feature_names)")
print("  print(result['stable_interactions'])")